In [11]:
!pip install litellm python-dotenv portkey-ai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [23]:
from dotenv import load_dotenv
import os

load_dotenv()


os.getenv("OPENAI_API_KEY")
os.getenv("ANTHROPIC_API_KEY")
os.getenv("GEMINI_API_KEY")

# 📌 Configure LiteLLM
import litellm

litellm.models_by_provider = {
    "openai": "gpt-4.1-mini",
    "anthropic": "claude-sonnet-4-5",
    "gemini": "gemini/gemini-2.5-flash"
}


In [15]:
# 📌 Helper function to call any provider
def run_llm(provider, prompt):
    model = litellm.models_by_provider[provider]
    print(f"\n▶ Provider: {provider} | Model: {model}")

    try:
        response = litellm.completion(
            model=model,
            messages=[{"role": "user", "content": prompt}],
        )
        print("Response:", response["choices"][0]["message"]["content"])
        return response
    except Exception as e:
        print("❌ Error:", e)
        return None

# 📌 Define test prompt
prompt = "Explain quantum computing in one sentence."

# 📌 Run experiment across all providers
results = {}

for provider in ["openai", "anthropic","gemini"]:
    results[provider] = run_llm(provider, prompt)



▶ Provider: openai | Model: gpt-4.1-mini
Response: Quantum computing is a type of computation that uses quantum bits, or qubits, which can exist in multiple states simultaneously, enabling certain complex calculations to be performed exponentially faster than classical computers.

▶ Provider: anthropic | Model: claude-sonnet-4-5
Response: Quantum computing uses quantum mechanical phenomena like superposition and entanglement to process information in ways that allow certain problems to be solved exponentially faster than classical computers.

▶ Provider: gemini | Model: gemini/gemini-2.5-flash
Response: Quantum computing leverages the principles of quantum mechanics—like superposition and entanglement—to process information using qubits, enabling it to tackle problems beyond the reach of classical computers.


In [9]:
# Prepare file
with open("sample_doc.txt", "w") as f:
    f.write("Quantum computing uses qubits to perform complex computations more efficiently.")

file_path = "sample_doc.txt"


File upload Enpoint only support : "openai", "azure", "vertex_ai", "bedrock", "hosted_vllm"

In [20]:
# Upload for OpenAI + Anthropic
uploaded_files = {}

for provider in ["openai", "anthropic"]:
    try:
        print(f"\nUploading for {provider}...")
        res = litellm.create_file(
            file=open("sample_doc.txt", "rb"),
            custom_llm_provider=provider,
            purpose="assistants"
        )
        uploaded_files[provider] = res
        print(res)
    except Exception as e:
        print(f"❌ {provider} upload failed:", e)



Uploading for openai...
id='file-MaY7xYgsogSBmv6BhQvqnC' bytes=79 created_at=1765438486 filename='sample_doc.txt' object='file' purpose='assistants' status='processed' expires_at=None status_details=None

Uploading for anthropic...
❌ anthropic upload failed: litellm.BadRequestError: LiteLLM doesn't support anthropic for 'create_file'. Only ['openai', 'azure', 'vertex_ai'] are supported.


/var/folders/hf/j2j8b0kj1n396dxxfkv9_nj80000gn/T/ipykernel_62455/573348641.py:7: RuntimeWarning: coroutine 'acreate_file' was never awaited
  res = litellm.create_file(


Create a vector store which can be used to store and search document chunks for retrieval-augmented generation (RAG) use cases.
https://docs.litellm.ai/docs/vector_stores/create

Only supports

Support LLM Providers (OpenAI /vector_stores API)	OpenAI	Full vector stores API support across providers
Support LLM Providers (Passthrough API)	Azure AI	Full vector stores API support across providers
Support LLM Providers (Dataset Management)	RAGFlow	Dataset creation and management (search not supported)


In [ ]:
vector_stores = {}

for provider in ["openai"]:
    try:
        print(f"\nCreating vector store for {provider}...")

        # No type hints available here
        store = litellm.vector_stores.create(
            name="My Document Store",
            file_ids=["file-MaY7xYgsogSBmv6BhQvqnC"]
        )
        vector_stores[provider] = store
        print(store)
    except Exception as e:
        print(f"❌ {provider} failed vector store creation:", e)



Creating vector store for gemini...

Provider List: https://docs.litellm.ai/docs/providers

{'id': 'vs_693a764fdd6c8191969ca0125ce7eb73', 'object': 'vector_store', 'created_at': 1765439056, 'name': 'My Document Store', 'description': None, 'usage_bytes': 0, 'file_counts': {'in_progress': 1, 'completed': 0, 'failed': 0, 'cancelled': 0, 'total': 1}, 'status': 'in_progress', 'expires_after': None, 'expires_at': None, 'last_active_at': 1765439056, 'metadata': {}}


https://docs.litellm.ai/docs/vector_stores/search

Search a vector store for relevant chunks based on a query and file attributes filter. 
Support LLM Providers	OpenAI, Azure OpenAI, Bedrock, Vertex RAG Engine, Azure AI, Milvus, Gemini

In [29]:
response = litellm.vector_stores.search(
    vector_store_id="vs_693a764fdd6c8191969ca0125ce7eb73",
    query="What is the capital of France?",
    custom_llm_provider="openai",
)
print(response)

{'object': 'vector_store.search_results.page', 'search_query': ['What is the capital of France?'], 'data': [{'file_id': 'file-MaY7xYgsogSBmv6BhQvqnC', 'filename': 'sample_doc.txt', 'score': 0.013189023074379266, 'attributes': {}, 'content': [{'type': 'text', 'text': 'Quantum computing uses qubits to perform complex computations more efficiently.'}]}], 'has_more': False, 'next_page': None}


https://docs.litellm.ai/docs/completion/knowledgebase



In [36]:

response = litellm.completion(
    model=litellm.models_by_provider["openai"],
    tools=[{
        "type": "file_search",
        "vector_store_ids": ["vs_693a764fdd6c8191969ca0125ce7eb73"]
    }],
    messages=[{"role": "user", "content": "What is the Quauntam Computing?"}],
)



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



BadRequestError: litellm.BadRequestError: OpenAIException - Invalid value: 'file_search'. Supported values are: 'function' and 'custom'.

Only Supported value for tool is 'function' and 'custom'.
That means we cannot use file_search_tool here, So we neeed to go with traditional RAG

In [ ]:

# This is method shown in litellm docs which is also not working
# https://docs.litellm.ai/docs/completion/knowledgebase
import os
import litellm

from litellm.vector_stores.vector_store_registry import VectorStoreRegistry, LiteLLM_ManagedVectorStore

# Init vector store registry
litellm.vector_store_registry = VectorStoreRegistry(
    vector_stores=[
        LiteLLM_ManagedVectorStore(
            vector_store_id="vs_693a764fdd6c8191969ca0125ce7eb73",
            custom_llm_provider="openai"
        )
    ]
)


# Make a completion request with vector_store_ids parameter
response = litellm.completion(
    model="anthropic/claude-3-5-sonnet", 
    messages=[{"role": "user", "content": "What is litellm?"}],
    tools=[
        {
            "type": "file_search",
            "vector_store_ids": ["T37J8R4WTM"]
        }
    ],
)

print(response)


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers



<frozen genericpath>:89: RuntimeWarning: coroutine 'acompletion' was never awaited


APIConnectionError: litellm.APIConnectionError: Unsupported tool type: file_search
Traceback (most recent call last):
  File "/Users/aviraj/Documents/timepass/.venv/lib/python3.12/site-packages/litellm/main.py", line 1377, in completion
    optional_params = get_optional_params(
                      ^^^^^^^^^^^^^^^^^^^^
  File "/Users/aviraj/Documents/timepass/.venv/lib/python3.12/site-packages/litellm/utils.py", line 3494, in get_optional_params
    optional_params = litellm.AnthropicConfig().map_openai_params(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/aviraj/Documents/timepass/.venv/lib/python3.12/site-packages/litellm/llms/anthropic/chat/transformation.py", line 694, in map_openai_params
    anthropic_tools, mcp_servers = self._map_tools(value)
                                   ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/aviraj/Documents/timepass/.venv/lib/python3.12/site-packages/litellm/llms/anthropic/chat/transformation.py", line 456, in _map_t


# Starting Experiment with Portkey

Portkey also does not support Tool Calling

In [42]:
from portkey_ai import Portkey

client = Portkey(
    api_key=os.getenv("PORTKEY_API_KEY"),
)


response = client.chat.completions.create(
    model="@openai/gpt-4o", 
    messages=[{"role": "user", "content": "What is litellm?"}],
    tools=[
        {
            "type": "file_search",
            "vector_store_ids": ["vs_693a764fdd6c8191969ca0125ce7eb73"]
        }
    ],
)

BadRequestError: Error code: 400 - {'error': {'message': "openai error: Invalid value: 'file_search'. Supported values are: 'function' and 'custom'.", 'type': 'invalid_request_error', 'param': 'tools[0].type', 'code': 'invalid_value'}, 'provider': 'openai'}